### <Center> Лабораторна робота №6. <br> Ідентифікація користувача за допомогою логістичної регресії

In [1]:
import pickle
import numpy as np
import pandas as pd
from tqdm import tqdm_notebook
from scipy.sparse import csr_matrix, hstack
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score
from sklearn.linear_model import LogisticRegression
%matplotlib inline
from matplotlib import pyplot as plt
import seaborn as sns

Спочатку налаштуємо доступ до даних на google drive (якщо ви відкриваєте блокнот в google colab, а не на PC) шляхом монтування google drive

In [ ]:
# from google.colab import drive
# drive.mount('/content/gdrive') 

Mounted at /content/gdrive


Перевіримо шлях до папки з матеріалами лаборатоної роботи на google drive. Якщо у вас шлях відрізняється то відредагуйте

In [ ]:
# !ls gdrive/'My Drive'/TEACHING/IntroDataScience/intro_to_data_science/Lab_5_6/data	

adult.data.csv	adult_train.csv  telecom_churn.csv  train_sessions.csv
adult_test.csv	site_dic.pkl	 test_sessions.csv


Перемістимо матеріали лабораторної роботи з google drive на віртуальну машину google colab

In [ ]:
# !cp -a gdrive/'My Drive'/TEACHING/IntroDataScience/intro_to_data_science/Lab_5_6/. .
# !ls

data				lab5_decision_trees.ipynb
gdrive				lab5_decision_trees.pdf
lab0_intro_decision_tree.ipynb	lab6_intruder_detection.ipynb
lab0_intro_decision_tree.pdf	sample_data


### 1. Завантаження і перетворення даних
Дані можна самостійно завантажити за посиланням [сторінка](https://inclass.kaggle.com/c/catch-me-if-you-can-intruder-detection-through-webpage-session-tracking2). Однак можна цього не робити, оскільки дані вже завантажені для проведення лабораторної роботи

In [2]:
# завантажимо навчальну і тестову вибірки
train_df = pd.read_csv('data/train_sessions.csv',
                       index_col='session_id')
test_df = pd.read_csv('data/test_sessions.csv',
                      index_col='session_id')

# приведемо колонку time1, ..., time10 до часового формату
times = ['time%s' % i for i in range(1, 11)]
train_df[times] = train_df[times].apply(pd.to_datetime)
test_df[times] = test_df[times].apply(pd.to_datetime)

# відсортуємо дані за часом
train_df = train_df.sort_values(by='time1')

# подивимося на заголовок навчальної вибірки
train_df.head()

,site1,time1,site2,time2,site3,time3,site4,time4,site5,time5,...,time6,site7,time7,site8,time8,site9,time9,site10,time10,target
session_id,,,,,,,,,,,,,,,,,,,,,
21669,56,2013-01-12 08:05:57,55.0,2013-01-12 08:05:57,NaN,NaT,NaN,NaT,NaN,NaT,...,NaT,NaN,NaT,NaN,NaT,NaN,NaT,NaN,NaT,0
54843,56,2013-01-12 08:37:23,55.0,2013-01-12 08:37:23,56.0,2013-01-12 09:07:07,55.0,2013-01-12 09:07:09,NaN,NaT,...,NaT,NaN,NaT,NaN,NaT,NaN,NaT,NaN,NaT,0
77292,946,2013-01-12 08:50:13,946.0,2013-01-12 08:50:14,951.0,2013-01-12 08:50:15,946.0,2013-01-12 08:50:15,946.0,2013-01-12 08:50:16,...,2013-01-12 08:50:16,948.0,2013-01-12 08:50:16,784.0,2013-01-12 08:50:16,949.0,2013-01-12 08:50:17,946.0,2013-01-12 08:50:17,0
114021,945,2013-01-12 08:50:17,948.0,2013-01-12 08:50:17,949.0,2013-01-12 08:50:18,948.0,2013-01-12 08:50:18,945.0,2013-01-12 08:50:18,...,2013-01-12 08:50:18,947.0,2013-01-12 08:50:19,945.0,2013-01-12 08:50:19,946.0,2013-01-12 08:50:19,946.0,2013-01-12 08:50:20,0
146670,947,2013-01-12 08:50:20,950.0,2013-01-12 08:50:20,948.0,2013-01-12 08:50:20,947.0,2013-01-12 08:50:21,950.0,2013-01-12 08:50:21,...,2013-01-12 08:50:21,946.0,2013-01-12 08:50:21,951.0,2013-01-12 08:50:22,946.0,2013-01-12 08:50:22,947.0,2013-01-12 08:50:22,0


В навчальній вибірці містяться наступні ознаки:
    - site1 – індекс першого відвідування сайту в сесії
    - time1 – час відвідування першого сайту в сесії
    - ...
    - site10 – індекс 10-го відвідування сайту в сесії
    - time10 – час відвідування 10-го сайту в сесії
    - target – цільова змінна, 1 для сесій Еліс, 0 для сесій інших користувачів
    
Сесії користувачів виділені таким чимном, щоб вони не можут бути довші півгодини чи 10 сайтів. Тобто сесія вважається закінченою або коли користувач відвідав 10 сайтів підряд або коли сесія зайняла за часом більше 30 хвилин.

В таблиці зустрічаються пропущені значення, це значить, що сесія містить менше, ніж 10 сайтів. Замінимо пропущені значення нулями і приведемо ознаки до цільового типу. Також заванатажимо словник сайтів і подивимося, як він виглядає:

In [3]:
# приведемо колонки site1, ..., site10 до цілочислового формату і замінимо пропуски нулями
sites = ['site%s' % i for i in range(1, 11)]
train_df[sites] = train_df[sites].fillna(0).astype('int')
test_df[sites] = test_df[sites].fillna(0).astype('int')

# завантажимо словник сайтів
with open(r"data/site_dic.pkl", "rb") as input_file:
    site_dict = pickle.load(input_file)

# датафрейм словника сайтів
sites_dict_df = pd.DataFrame(list(site_dict.keys()), 
                          index=list(site_dict.values()), 
                          columns=['site'])
print(u'всього сайтів:', sites_dict_df.shape[0])
sites_dict_df.head()

всього сайтів: 48371


,site
25075,www.abmecatronique.com
13997,groups.live.com
42436,majeureliguefootball.wordpress.com
30911,cdt46.media.tourinsoft.eu
8104,www.hdwallpapers.eu


Виділимо цільову змінну і об'єднаємо вибірки, щоб разом привести їх до розрідженого формату.

In [4]:
# наша цільова змінна
y_train = train_df['target']

# об'єднана таблиця вхідних даних
full_df = pd.concat([train_df.drop('target', axis=1), test_df])

# індекс, за яким будемо відокремлювати навчальну вибірку від тестової
idx_split = train_df.shape[0]

Для самої першої моделі ми використовуємо лише відвідувані сайти в сесіях (але не будемо звертати увагу на часові ознаки). В основі такого вибору даних для моделей лежить така ідея: * у Еліс є свої улюблені сайти, і якщо ви ще побачите ці сайти в сесіях, тим вище ймовірність, що це сесія Еліс і навпаки. *

Підготуємо дані, з усієї таблиці виберемо лише ознаки `site1, site2, ..., site10`. Нагадуємо, що пропущені значення замінені нулем. Ось як виглядатимуть перші рядки таблиць:

In [5]:
# таблиця з індексами відвіданих сайтів в сесії
full_sites = full_df[sites]
full_sites.head()

,site1,site2,site3,site4,site5,site6,site7,site8,site9,site10
session_id,,,,,,,,,,
21669,56,55,0,0,0,0,0,0,0,0
54843,56,55,56,55,0,0,0,0,0,0
77292,946,946,951,946,946,945,948,784,949,946
114021,945,948,949,948,945,946,947,945,946,946
146670,947,950,948,947,950,952,946,951,946,947


Сесії представляють собою послідовність індексів сайтів і дані в такому вигляді невдалі для лінійних методів. Відповідно до нашої гіпотези (у Еліс є улюблені сайти) необхідно перетворити цю таблицю таким чином, щоб кожен можливий веб-сайт відповідав своєму окремому призначенню (колонка), а його значення зростало за кількістю відвідувачів цього веб-сайту в сесіях. Це робиться в два рядки:

In [6]:
from scipy.sparse import csr_matrix

In [7]:
csr_matrix?

Init signature: csr_matrix(arg1, shape=None, dtype=None, copy=False)
Docstring:     
Compressed Sparse Row matrix.

This can be instantiated in several ways:
    csr_matrix(D)
        where D is a 2-D ndarray

    csr_matrix(S)
        with another sparse array or matrix S (equivalent to S.tocsr())

    csr_matrix((M, N), [dtype])
        to construct an empty matrix with shape (M, N)
        dtype is optional, defaulting to dtype='d'.

    csr_matrix((data, (row_ind, col_ind)), [shape=(M, N)])
        where ``data``, ``row_ind`` and ``col_ind`` satisfy the
        relationship ``a[row_ind[k], col_ind[k]] = data[k]``.

    csr_matrix((data, indices, indptr), [shape=(M, N)])
        is the standard CSR representation where the column indices for
        row i are stored in ``indices[indptr[i]:indptr[i+1]]`` and their
        corresponding values are stored in ``data[indptr[i]:indptr[i+1]]``.
        If the shape parameter is not supplied, the matrix dimensions
        are inferred from 

In [8]:
# послідовність з індексами
sites_flatten = full_sites.values.flatten()

# шкана матриця
full_sites_sparse = csr_matrix(([1] * sites_flatten.shape[0],
                                sites_flatten,
                                range(0, sites_flatten.shape[0] + 10, 10)))[:, 1:]

Ще один плюс використання розріджених матриць у тому, що для них є спеціальні реалізації як матричних операцій, так і алгоритми машинного навчання, що дозволяє істотно прискорити операції за рахунок особливостей структур даних. Це стосується і логістичної регресії. Ось тепер у нас все готове для побудови наших перших моделей.

### 2. Побудова першої моделі

Отже, у нас є алгоритм та дані для нього, побудуйте нашу першу модель, використовуючи реалізацію [логістичної регресії] (http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html) з пакета ` sklearn` з параметрами за замовчуванням. Перші 90% даних будемо використовувати для навчання (навчальна вибірка, відсортована за часом), а також 10% для перевірки якості (validation).

**Напишіть просту функцію, яка поверне якість моделей на вкладеній вибірці, і навчіть наш перший класифікатор**.

In [13]:
def get_auc_lr_valid(X, y, C=1.0, ratio = 0.9, seed=17):
    '''
    X, y – вибірка
    ratio – у якому співвідношенні поділити вибірку
    C, seed – коефіцієт регуляризації і random_state 
              логістичної регресії
    '''
    
    # Розділимо дані на навчальну та валідаційну вибірки
    idx = int(X.shape[0] * ratio)
    X_train, X_valid = X[:idx, :], X[idx:, :]
    y_train, y_valid = y[:idx], y[idx:]

    # Ініціалізуємо та навчаємо модель логістичної регресії
    logit = LogisticRegression(C=C, random_state=seed, solver='liblinear')
    logit.fit(X_train, y_train)

    # Прогнозуємо ймовірності для валідаційної вибірки
    y_pred = logit.predict_proba(X_valid)[:, 1]

    # Обчислюємо та повертаємо значення метрики ROC AUC
    return roc_auc_score(y_valid, y_pred)

**Подивіться, який отримано ROC AUC на відкладеній вибірці.**

In [14]:
# Обчислимо ROC AUC на відкладеній вибірці
roc_auc = get_auc_lr_valid(full_sites_sparse[:idx_split], y_train)
print(f'ROC AUC на відкладеній вибірці: {roc_auc:.4f}')

ROC AUC на відкладеній вибірці: 0.9195


Будем вважати цю модель нашою першою відправною точкою (базовий рівень). Для побудови моделей для прогнозування на тестовій вибірці ** необхідно навчити модель заново вже на всій навчальній вибірці ** (покищо наша модель навчилася лише на частині даних), що підвищує її узагальнюючу здатність:

In [15]:
# функція для запису прогнозів в файлі
def write_to_submission_file(predicted_labels, out_file,
                             target='target', index_label="session_id"):
    predicted_df = pd.DataFrame(predicted_labels,
                                index = np.arange(1, predicted_labels.shape[0] + 1),
                                columns=[target])
    predicted_df.to_csv(out_file, index_label=index_label)

**Навчіть модель на всій вибірці, зробіть прогноз для тестової вибірки і покажіть результат**.

In [16]:
# Ініціалізуємо та навчаємо модель логістичної регресії на всій навчальній вибірці
logit = LogisticRegression(C=1.0, random_state=17, solver='liblinear')
logit.fit(full_sites_sparse[:idx_split], y_train)

# Робимо прогнози для тестової вибірки
test_pred = logit.predict_proba(full_sites_sparse[idx_split:])[:, 1]

# Записуємо прогнози у файл для подання
write_to_submission_file(test_pred, 'submission.csv')

# Відображаємо перші кілька прогнозів
print(test_pred[:10])

[2.21002459e-03 4.81001814e-09 1.87251238e-08 2.35459752e-08
 3.13038690e-05 2.18462626e-04 5.47933060e-04 1.32278952e-04
 7.95194308e-04 1.03130297e-01]


### 3. Покращення моделі, побудова нових ознак

Створіть таку ознаку, яка буде представлят собою число формату ГГГГММ від тої дати, коли відбувалась сесія, наприклад 201407 -- 2014 рік і 7 месяц. Таким чином, ми будемо враховувати помісячний [линейный тренд](http://people.duke.edu/~rnau/411trend.htm) за весь період наданих даних.

In [17]:
# Створимо нову ознаку 'start_month' у форматі ГГГГММ
full_df['start_month'] = full_df['time1'].dt.year * 100 + full_df['time1'].dt.month

# Відобразимо перші кілька рядків, щоб перевірити нову ознаку
full_df[['time1', 'start_month']].head()

,time1,start_month
session_id,,
21669,2013-01-12 08:05:57,201301
54843,2013-01-12 08:37:23,201301
77292,2013-01-12 08:50:13,201301
114021,2013-01-12 08:50:17,201301
146670,2013-01-12 08:50:20,201301


Додайте нову ознаку, попередньо нормалізуючи її за допомогою `StandardScaler`, і знову підрахуйте ROC AUC на відкладеній вибірці.

In [18]:
from sklearn.preprocessing import StandardScaler

# Нормалізуємо ознаку 'start_month'
scaler = StandardScaler()
start_month_scaled = scaler.fit_transform(full_df[['start_month']])

# Перетворюємо нормалізовану ознаку в розріджену матрицю
start_month_sparse = csr_matrix(start_month_scaled)

# Об'єднуємо початкову розріджену матрицю з новою ознакою
full_sites_with_month = hstack([full_sites_sparse, start_month_sparse])

# Перераховуємо ROC AUC на відкладеній вибірці
roc_auc_with_month = get_auc_lr_valid(full_sites_with_month[:idx_split], y_train)
print(f'ROC AUC з ознакою start_month: {roc_auc_with_month:.4f}')


ROC AUC з ознакою start_month: 0.9197


**Додайте дві нові ознаки: start_hour і morning.**

Ознака `start_hour` - це час у якому почалася сесія (від 0 до 23), а бінарна оознака` morning` рівна 1, якщо сесія почалася вранці і 0, якщо сесія почалася пізніше (будемо вважати, що це ранок, якщо `start_hour рівний` 11 або менше).

**Підрахуйте RUC AUC на відкладеній вибірці для вибірки з:**
- сайтами, `start_month` і` start_hour`
- сайтами, `start_month` і` morning`
- сайтами, `start_month`,` start_hour` і `morning`

In [19]:
# Додамо нові ознаки 'start_hour' і 'morning'
full_df['start_hour'] = full_df['time1'].dt.hour
full_df['morning'] = (full_df['start_hour'] <= 11).astype(int)

# Нормалізуємо нові ознаки
start_hour_scaled = scaler.fit_transform(full_df[['start_hour']])
morning_scaled = scaler.fit_transform(full_df[['morning']])

# Перетворюємо нормалізовані ознаки в розріджені матриці
start_hour_sparse = csr_matrix(start_hour_scaled)
morning_sparse = csr_matrix(morning_scaled)

# Об'єднуємо початкову розріджену матрицю з новими ознаками
full_sites_with_hour = hstack([full_sites_with_month, start_hour_sparse])
full_sites_with_morning = hstack([full_sites_with_month, morning_sparse])
full_sites_with_all = hstack([full_sites_with_month, start_hour_sparse, morning_sparse])

# Перераховуємо ROC AUC на відкладеній вибірці для різних комбінацій ознак
roc_auc_with_hour = get_auc_lr_valid(full_sites_with_hour[:idx_split], y_train)
roc_auc_with_morning = get_auc_lr_valid(full_sites_with_morning[:idx_split], y_train)
roc_auc_with_all = get_auc_lr_valid(full_sites_with_all[:idx_split], y_train)

print(f'ROC AUC з ознаками start_month і start_hour: {roc_auc_with_hour:.4f}')
print(f'ROC AUC з ознаками start_month і morning: {roc_auc_with_morning:.4f}')
print(f'ROC AUC з ознаками start_month, start_hour і morning: {roc_auc_with_all:.4f}')

ROC AUC з ознаками start_month і start_hour: 0.9579
ROC AUC з ознаками start_month і morning: 0.9488
ROC AUC з ознаками start_month, start_hour і morning: 0.9592


### 4. Підбір коефіцієнта регуляризації

Отже, ми ввели ознаки, які покращують якість нашої моделі у порівнянні з першим бейслайном. Чи можемо ми домогтися більшого значення метрики? Після того, як ми сформували навчальну та тестову вибірки, майже завжди має сенс підібрати оптимальні гіперпараметри - характеристики моделі, які не змінюються під час навчання. Наприклад, ви вивчали вирішальні дерева, глибина дерева це гіперпараметр, а ознака, за якому відбувається розгалуження і її значення - ні. У використовуваної нами логістичної регресії ваги кожної ознаки змінюються і під час навчання знаходяться їх оптимальні значення, а коефіцієнт регуляризації залишається постійним. Це той гіперпараметр, який ми зараз будемо оптимізувати.

Порахуйте якість на відкладеній вибірці з коефіцієнтом регуляризації, який за замовчуванням `C = 1 ':

In [20]:
# Обчислимо якість на відкладеній вибірці з коефіцієнтом регуляризації C = 1
roc_auc_default_C = get_auc_lr_valid(full_sites_with_all[:idx_split], y_train, C=1.0)
print(f'ROC AUC на відкладеній вибірці з C=1: {roc_auc_default_C:.4f}')

ROC AUC на відкладеній вибірці з C=1: 0.9592


Постараємося побити цей результат за рахунок оптимізації коефіцієнта регуляризації. Візьмемо набір можливих значень C і для кожного з них порахуємо значення метрики на відкладеної вибірці.

Знайдіть `C` з` np.logspace (-3, 1, 10) `, при якому ROC AUC на відкладеної вибірці максимальний.

In [21]:
# Визначимо діапазон значень C для тестування
C_values = np.logspace(-3, 1, 10)

# Ініціалізуємо змінні для збереження найкращого C і відповідного ROC AUC
best_C = None
best_roc_auc = 0

# Ітеруємо по діапазону значень C
for C in C_values:
    # Обчислюємо ROC AUC для поточного значення C
    roc_auc = get_auc_lr_valid(full_sites_with_all[:idx_split], y_train, C=C)
    print(f'ROC AUC для C={C}: {roc_auc:.4f}')
    
    # Оновлюємо найкраще значення C і ROC AUC, якщо поточне краще
    if roc_auc > best_roc_auc:
        best_C = C
        best_roc_auc = roc_auc

print(f'Найкраще C: {best_C}')
print(f'Найкращий ROC AUC: {best_roc_auc:.4f}')

ROC AUC для C=0.001: 0.8230
ROC AUC для C=0.0027825594022071257: 0.8965
ROC AUC для C=0.007742636826811269: 0.9390
ROC AUC для C=0.021544346900318832: 0.9564
ROC AUC для C=0.05994842503189409: 0.9607
ROC AUC для C=0.1668100537200059: 0.9612
ROC AUC для C=0.46415888336127775: 0.9603
ROC AUC для C=1.2915496650148828: 0.9587
ROC AUC для C=3.593813663804626: 0.9558
ROC AUC для C=10.0: 0.9513
Найкраще C: 0.1668100537200059
Найкращий ROC AUC: 0.9612


Нарешті, навчіть модель зі знайденим оптимальним значенням коефіцієнта регуляризації і з побудованими ознаками `start_hour`,` start_month` і `morning`. 

In [22]:
# Ініціалізуємо та навчаємо модель логістичної регресії з оптимальним значенням C
logit_optimal = LogisticRegression(C=best_C, random_state=17, solver='liblinear')
logit_optimal.fit(full_sites_with_all[:idx_split], y_train)

# Робимо прогнози для тестової вибірки
test_pred_optimal = logit_optimal.predict_proba(full_sites_with_all[idx_split:])[:, 1]

# Записуємо прогнози у файл для подання
write_to_submission_file(test_pred_optimal, 'submission_optimal.csv')

# Відображаємо перші кілька прогнозів
print(test_pred_optimal[:10])

[3.28563751e-04 2.85290394e-06 9.31028300e-07 6.40058664e-07
 1.98830422e-04 7.07909326e-05 2.94006998e-03 1.85383443e-03
 1.33196723e-04 3.15635042e-03]
